In [1]:
%pip install --upgrade --force-reinstall transformers==4.52.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 142.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 134.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.2/507.2 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
%pip install -q transformers datasets scikit-learn pandas accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.9 MB/s eta 0:00:00


In [2]:
from transformers import TrainingArguments

In [3]:
args = TrainingArguments(output_dir="./results")
# print(args)


In [4]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.metrics import classification_report

In [5]:
!pip install openpyxl


In [6]:
from google.colab import files
uploaded=files.upload()

Saving faza_1_training_dataset_sa_comma (1).csv to faza_1_training_dataset_sa_comma (1).csv


In [7]:
df=pd.read_csv("faza_1_training_dataset_sa_comma (1).csv")

In [8]:
df


,surveyId,resultId,surveyYear,sentiment,questionId,questionText,questionAnswer,questionType,usergroupType,category
0,238,77062,2023,positive,554,Warum würden Sie Ihre Bank im Bekanntenkreis a...,"sicher Arbeitsplatz, gutes Arbeitsklima, gute ...",Text,managers,"Entlohnung, Grundstimmung, Zukunftsperspektive..."
1,238,77062,2023,positive,558,Woran machen Sie Ihre Bewertung fest?,"sicher Arbeitsplatz, tolles Team, gutes Image...",Text,managers,"Zukunftsperspektive/Arbeitsplatzsicherheit, gu..."
2,238,77062,2023,positive,2030,Was gefällt Ihnen an unserer Kultur besonders ...,Wertschätzung gegenüber den Mitarbeitern.,Text,managers,Wertschätzung
3,238,77062,2023,positive,2118,"Warum würden Sie es weiterempfehlen, in Ihrem ...","tolles Team, interessanter Aufgabenbereich",Text,managers,"interessant/spannend, tolles Team"
4,238,77064,2023,negative,547,Durch welche Maßnahmen ließe sich das Arbeitsk...,"- schwer zu sagen, denn, dass sich jemand pers...",Text,managers,Eigenverantwortung
...,...,...,...,...,...,...,...,...,...,...
24646,531,144319,2025,negative,2610,Angenommen Sie könnten sich von Jelica Trivano...,Ich habe keine Verbesserungsvorschläge,Text,managers,Alles gut
24647,531,144322,2025,positive,2611,Was möchten Sie ergänzend Jelica Trivanovic no...,Setzt sich gut für das Team ein. Das schätze i...,Text,managers,die Kolleg:innen
24648,549,145052,2026,negative,2908,Was fällt dir bei SB-Technik & Bargeld auf – w...,SB-Technik,Text,managers,Technik
24649,549,145052,2026,neutral,2907,Was fällt dir beim Beratungserlebnis auf – was...,Beratungserlebnis,Text,managers,Beratung


In [9]:
df=df[["questionText", "questionAnswer","category"]]

In [10]:
df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
df['labels'] = df['category'].apply(lambda x: [c.strip() for c in x.split(',')])

In [11]:
# Drop rows where the 'text' column is NaN
df = df.dropna(subset=["questionText", "questionAnswer", "labels"])

# Optionally, reset the index after dropping rows (if needed)
df = df.reset_index(drop=True)

In [12]:
import numpy as np

df["questionAnswer"] = df["questionAnswer"].replace("#NAME?", np.nan)

df = df.dropna(subset=["questionAnswer"]).copy()
df = df[df["questionAnswer"].astype(str).str.strip().ne("")].copy()

df = df.reset_index(drop=True)

In [13]:
df=df[["questionText", "questionAnswer","labels"]]

In [14]:
!pip install iterative-stratification


In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

df = df.reset_index(drop=True)

# (optional but recommended) normalize answer so Nein/nein match
df["questionAnswer_norm"] = (
    df["questionAnswer"]
      .astype(str)
      .str.strip()
      .str.replace(r"\s+", " ", regex=True)
      .str.lower()
)

# 1) Create group id for each unique (questionText, normalized answer)
df["pair_id"] = pd.util.hash_pandas_object(
    df[["questionText", "questionAnswer_norm"]],
    index=False
).astype("int64")

group_col = "pair_id"

# 2) Build ONE label-list per group (union of labels inside the group)
grouped = (
    df.groupby(group_col)["labels"]
      .apply(lambda s: sorted(set(l for labs in s for l in labs)))
      .reset_index()
)

groups = grouped[group_col].values
group_labels_list = grouped["labels"]

# 3) Multi-hot for stratifying groups
mlb_groups = MultiLabelBinarizer()
Y_groups = mlb_groups.fit_transform(group_labels_list)

# --- Split groups: trainval vs test (10%) ---
msss1 = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.10, random_state=42)
trainval_g_idx, test_g_idx = next(msss1.split(np.zeros(len(groups)), Y_groups))

trainval_groups = set(groups[trainval_g_idx])
test_groups     = set(groups[test_g_idx])

df_trainval = df[df[group_col].isin(trainval_groups)].copy()
df_test     = df[df[group_col].isin(test_groups)].copy()

# --- Split trainval groups: train vs val (val = 10% total => 0.111111 of trainval) ---
grouped_trainval = grouped[grouped[group_col].isin(trainval_groups)].reset_index(drop=True)
groups_tv = grouped_trainval[group_col].values
labels_tv = grouped_trainval["labels"]

mlb_tv = MultiLabelBinarizer()
Y_tv = mlb_tv.fit_transform(labels_tv)

msss2 = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.111111, random_state=42)
train_g_idx, val_g_idx = next(msss2.split(np.zeros(len(groups_tv)), Y_tv))

train_groups = set(groups_tv[train_g_idx])
val_groups   = set(groups_tv[val_g_idx])

df_train = df_trainval[df_trainval[group_col].isin(train_groups)].copy()
df_val   = df_trainval[df_trainval[group_col].isin(val_groups)].copy()

# --- 4) Build X ---
X_train = df_train[["questionText", "questionAnswer"]]
X_val   = df_val[["questionText", "questionAnswer"]]
X_test  = df_test[["questionText", "questionAnswer"]]

# --- 5) Fit MultiLabelBinarizer on TRAIN ONLY (clean) ---
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train["labels"])
y_val   = mlb.transform(df_val["labels"])
y_test  = mlb.transform(df_test["labels"])

# Optional: y as DataFrames
y_train = pd.DataFrame(y_train, columns=mlb.classes_, index=df_train.index)
y_val   = pd.DataFrame(y_val,   columns=mlb.classes_, index=df_val.index)
y_test  = pd.DataFrame(y_test,  columns=mlb.classes_, index=df_test.index)

print(len(df_train), len(df_val), len(df_test))
print("Num labels:", len(mlb.classes_))
print("Labels missing from train:", set().union(*df["labels"]) - set().union(*df_train["labels"]))


19764 2427 2460
Num labels: 211
Labels missing from train: set()


In [16]:
df_test.to_csv("df_test.csv", index=False, encoding="utf-8")
print("Saved df_test.csv")
from google.colab import files
files.download("df_test.csv")

Saved df_test.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
# 4. Tokenization (Option B: German model)
from transformers import AutoTokenizer
import torch

checkpoint = "deepset/gbert-base"   # German BERT (recommended for German data)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_texts(text_a, text_b, labels):
    encodings = tokenizer(
        text=text_a.tolist(),
        text_pair=text_b.tolist(),
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )
    encodings["labels"] = torch.tensor(labels, dtype=torch.float32)
    return encodings

# Train encodings
train_encodings = tokenize_texts(
    X_train["questionText"],
    X_train["questionAnswer"],
    y_train.values
)

# Validation encodings
val_encodings = tokenize_texts(
    X_val["questionText"],
    X_val["questionAnswer"],
    y_val.values
)

# Test encodings
test_encodings = tokenize_texts(
    X_test["questionText"],
    X_test["questionAnswer"],
    y_test.values
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [18]:
class SurveyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = SurveyDataset(train_encodings)
val_dataset = SurveyDataset(val_encodings)
test_dataset = SurveyDataset(test_encodings)

In [19]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "deepset/gbert-base",
    num_labels=y_train.shape[1],
    problem_type="multi_label_classification"
)


model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Freeze everything, then unfreeze top 4 encoder layers + pooler + classifier
# Works with BertForSequenceClassification / AutoModelForSequenceClassification (BERT-style)

def freeze_all_then_unfreeze_top_k_bert(model, k=12):
    # 1) freeze all params
    for p in model.parameters():
        p.requires_grad = False

    # 2) unfreeze top-k encoder layers (BERT-base has 12 layers: 0..11)
    top_layers = list(range(12 - k, 12))  # e.g. k=4 -> [8,9,10,11]

    for name, p in model.named_parameters():
        # encoder layers
        if any(f"encoder.layer.{i}." in name for i in top_layers):
            p.requires_grad = True

        # optional: pooler (small, often helps)
        if "pooler" in name:
            p.requires_grad = True

        # classifier head
        if "classifier" in name:
            p.requires_grad = True

    # 3) print trainable params
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"Unfroze top {k} layers + pooler + classifier")
    print(f"Trainable params: {trainable:,} / {total:,} ({trainable/total:.2%})")

# --- use it ---
freeze_all_then_unfreeze_top_k_bert(model, k=12)


Unfroze top 12 layers + pooler + classifier
Trainable params: 85,807,315 / 110,089,939 (77.94%)


In [21]:
import numpy as np
import torch

# y_train: DataFrame or numpy array with shape (n_samples, n_labels), values {0,1}
Y = y_train.values if hasattr(y_train, "values") else y_train

pos = Y.sum(axis=0)                 # positives per label
neg = Y.shape[0] - pos              # negatives per label

# avoid division by zero (shouldn't happen if every label exists in train)
pos_weight = neg / np.clip(pos, 1, None)

pos_weight_t = torch.tensor(pos_weight, dtype=torch.float32)
print("pos_weight shape:", pos_weight_t.shape)  # (num_labels,)
print("pos_weight min/median/max:", pos_weight_t.min().item(), np.median(pos_weight), pos_weight_t.max().item())


pos_weight shape: torch.Size([211])
pos_weight min/median/max: 11.007290840148926 204.875 493.1000061035156


In [22]:
from transformers import Trainer
import torch
import torch.nn as nn

class WeightedMultilabelTrainer(Trainer):
    def __init__(self, *args, pos_weight=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight  # torch tensor shape (num_labels,)

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels").float()  # ensure float32
        outputs = model(**inputs)
        logits = outputs.logits

        # Make sure pos_weight is on same device as logits
        if self.pos_weight is not None:
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight.to(logits.device))
        else:
            loss_fct = nn.BCEWithLogitsLoss()

        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [23]:
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_metrics(eval_pred, threshold=0.5, force_top1=False):
    logits, y_true = eval_pred  # shapes: (N, C)
    probs = sigmoid(logits)

    # threshold-based multi-label prediction
    y_pred = (probs >= threshold).astype(int)

    # Optional: guarantee at least one label per sample (useful for your goal)
    if force_top1:
        empty = (y_pred.sum(axis=1) == 0)
        if np.any(empty):
            top1 = probs[empty].argmax(axis=1)
            y_pred[empty] = 0
            y_pred[empty, top1] = 1

    # --- Coverage / emptiness metrics ---
    pred_counts = y_pred.sum(axis=1)                    # how many labels predicted per sample
    coverage = np.mean(pred_counts > 0)                # fraction with at least 1 label
    avg_labels = float(np.mean(pred_counts))           # label cardinality (avg predicted labels per sample)


    return {
        "f1_micro": f1_score(y_true, y_pred, average="micro", zero_division=0),
        "f1_macro": f1_score(y_true, y_pred, average="macro", zero_division=0),
        "precision_micro": precision_score(y_true, y_pred, average="micro", zero_division=0),
        "recall_micro": recall_score(y_true, y_pred, average="micro", zero_division=0),

        # new: usefulness/behavior metrics
        "coverage_at_least_one": float(coverage),
        "avg_pred_labels": avg_labels,
    }


In [24]:
from transformers import TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./output_model",

    eval_strategy="epoch",         # your transformers version uses this
    save_strategy="epoch",

    learning_rate=1e-5,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
    weight_decay=0.01,

    per_device_train_batch_size=16,   # H100 can handle this usually
    per_device_eval_batch_size=32,
    num_train_epochs=15,               # early stopping will stop earlier if needed

    bf16=True,                        # ✅ best on H100
    dataloader_pin_memory=True,       # fine on GPU
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,

    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    greater_is_better=True,

    report_to=[],
)


trainer = WeightedMultilabelTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # fixed threshold metrics during training
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    pos_weight=pos_weight_t,          # ✅ your computed pos_weight tensor
)

/tmp/ipython-input-1173149034.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedMultilabelTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,Precision Micro,Recall Micro,Coverage At Least One,Avg Pred Labels
1,1.335500,1.259681,0.029855,0.031718,0.015220,0.776673,1.000000,93.017717
2,0.945000,0.925737,0.058888,0.056654,0.030446,0.894213,1.000000,53.536465
3,0.736600,0.735390,0.091755,0.085476,0.048330,0.904159,1.000000,34.101772
4,0.615100,0.624138,0.120193,0.110530,0.064369,0.905289,0.998764,25.636176


In [ ]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def find_best_thresholds(logits, y_true, thresholds=np.arange(0.1, 1.00, 0.01)):
    probs = sigmoid(logits)
    num_labels = y_true.shape[1]
    best_thresholds = np.full(num_labels, 0.5, dtype=float)

    for i in range(num_labels):
        best_f1, best_t = -1.0, 0.5
        yt = y_true[:, i]
        pr = probs[:, i]
        for t in thresholds:
            pred_i = (pr >= t).astype(int)
            f1 = f1_score(yt, pred_i, zero_division=0)
            if f1 > best_f1:
                best_f1, best_t = f1, t
        best_thresholds[i] = best_t

    return best_thresholds

def eval_with_thresholds(logits, y_true, thresholds):
    probs = sigmoid(logits)
    y_pred = (probs >= thresholds).astype(int)
    # --- Coverage / emptiness metrics ---
    pred_counts = y_pred.sum(axis=1)                    # how many labels predicted per sample
    coverage = np.mean(pred_counts > 0)                # fraction with at least 1 label
    avg_labels = float(np.mean(pred_counts))           # label cardinality (avg predicted labels per sample)
    return {
        "f1_micro": f1_score(y_true, y_pred, average="micro", zero_division=0),
        "f1_macro": f1_score(y_true, y_pred, average="macro", zero_division=0),
        "precision_micro": precision_score(y_true, y_pred, average="micro", zero_division=0),
        "recall_micro": recall_score(y_true, y_pred, average="micro", zero_division=0),
                # new: usefulness/behavior metrics
        "coverage_at_least_one": float(coverage),
        "avg_pred_labels": avg_labels,
    }

# Tune thresholds on VAL once
val_pred = trainer.predict(val_dataset)
best_thresholds = find_best_thresholds(val_pred.predictions, val_pred.label_ids)

# Evaluate TEST with frozen thresholds
test_pred = trainer.predict(test_dataset)
metrics_test = eval_with_thresholds(test_pred.predictions, test_pred.label_ids, best_thresholds)
print(metrics_test)


In [ ]:
import os
import json
import shutil
import numpy as np

# ===== Where to save =====
local_dir = "./my_trained_model"
os.makedirs(local_dir, exist_ok=True)

# ===== 1) Save best model + tokenizer + training args =====
trainer.save_model(local_dir)                 # saves model + config
tokenizer.save_pretrained(local_dir)          # saves tokenizer files

with open(os.path.join(local_dir, "training_args.json"), "w") as f:
    f.write(trainer.args.to_json_string())

# ===== 2) Save label names =====
label_names = None
if hasattr(trainer.model, "config") and getattr(trainer.model.config, "id2label", None):
    id2label = trainer.model.config.id2label

    # handle id2label keys being strings or ints
    keys = sorted([int(k) for k in id2label.keys()])
    label_names = [id2label[str(k)] if str(k) in id2label else id2label[k] for k in keys]
else:
    raise ValueError("model.config.id2label missing — please provide label_names manually.")

with open(os.path.join(local_dir, "label_names.json"), "w") as f:
    json.dump(label_names, f, indent=2)

# ===== 3) Save thresholds you computed (VAL-tuned, frozen for TEST) =====
# You already computed `best_thresholds` earlier with find_best_thresholds(...)
# Ensure it's JSON-serializable:
best_thresholds_list = np.asarray(best_thresholds, dtype=float).tolist()

with open(os.path.join(local_dir, "best_thresholds.json"), "w") as f:
    json.dump(best_thresholds_list, f, indent=2)

# ===== 4) Save useful metadata =====
metadata = {
    "metric_for_best_model": trainer.args.metric_for_best_model,
    "best_metric": trainer.state.best_metric,
    "global_step": trainer.state.global_step,
    "num_train_epochs": trainer.state.epoch,
}
with open(os.path.join(local_dir, "metadata.json"), "w") as f:
    json.dump(metadata, f, indent=2)

# Optional: save log history
with open(os.path.join(local_dir, "log_history.json"), "w") as f:
    json.dump(trainer.state.log_history, f, indent=2)

# ===== 5) Zip + download =====
zip_path = shutil.make_archive("my_trained_model", "zip", local_dir)
print(f"📦 Zipped to: {zip_path}")

# Colab download (if in Colab)
try:
    from google.colab import files
    files.download(zip_path)
except Exception:
    print("✅ Not running in Colab — zip saved locally.")


In [ ]:
print("Best checkpoint:", trainer.state.best_model_checkpoint)
print("Best metric:", trainer.state.best_metric)
print("num_labels in config:", trainer.model.config.num_labels)
print("thresholds length:", len(best_thresholds))


In [ ]:
import json
import numpy as np

with open("./my_trained_model/best_thresholds.json") as f:
    reloaded = np.array(json.load(f), dtype=float)

print("same values:", np.allclose(reloaded, np.asarray(best_thresholds, dtype=float)))
print("min/max thresholds:", reloaded.min(), reloaded.max())


In [ ]:
# import json

# # Load log history from file
# with open("./my_trained_model/log_history.json", "r") as f:
#     log_history = json.load(f)

# # Print evaluation entries (with metrics like F1, precision, thresholds, etc.)
# print("📊 Evaluation Metrics per Epoch:\n")
# for entry in log_history:
#     if "eval_loss" in entry:  # Only show evaluation steps
#         print(f"Epoch {entry.get('epoch', '?')} (Step {entry.get('step', '?')}):")
#         print(f"  - Eval Loss         : {entry.get('eval_loss'):.4f}")
#         print(f"  - F1 Micro          : {entry.get('eval_f1_micro'):.4f}")
#         print(f"  - F1 Macro          : {entry.get('eval_f1_macro'):.4f}")
#         print(f"  - Precision (Micro) : {entry.get('eval_precision_micro'):.4f}")
#         print(f"  - Recall (Micro)    : {entry.get('eval_recall_micro'):.4f}")
#         print(f"  - Thresholds (len)  : {len(entry.get('eval_best_thresholds', []))} labels")
#         print("-" * 50)
